# Esame di Metodi Numerici - 6 Luglio 2023
## Turno II -- Ore 11.45

## Esercizio 1


Nel file ``Test_II.mat`` sono memorizzate le matrici A1, A2 ed A3 ed i vettori b1,b2,b3. Risolvere i 3 sistemi lineari aventi ciascuno di essi come matrice dei coefficienti A1 e termine noto b1, A2 e termine noto b2, A3 e terimine noto b3  utilizzando il metodo più adatto per ciascuno di essi e  commentare i risultati ottenuti  giustificandoli alla luce della teoria.
Verificare se le matrici sono malcondizionate, dire teoricamente cosa questo implica e verificarlo sperimentalmente.


Per la lettura dei dati procedere nel seguente modo:

``from scipy.io import loadmat``

``import numpy as np``

``dati = loadmat('Test_II.mat')``

``A1=dati["A1"] ``

``A1=A1.astype(float)``

`` b1=dati["b1"] ``

`` b1=b1.astype(float)``

``A2=dati["A2"] ``

``A2=A2.astype(float)``

`` b2=dati["b2"] ``

`` b2=b2.astype(float)``

``A3=dati["A3"] ``

``A3=A3.astype(float)``

`` b3=dati["b3"] ``

`` b3=b3.astype(float)``



                                                                    Punti 16

In [382]:
import numpy as np
def cond(A):
    condi=np.linalg.cond(A)
    if condi < 10**2:
        print("matrice ben condizionata")
    elif condi >= 10**2 and condi < 10**4:
        print("matrice mediamente condizionata")
    else:
         print("matrice mal condizionata")

def checkMaxRank(A):
    rank = np.linalg.matrix_rank(A)
    if rank == min(A.shape):
        print("max rank")
    else : 
        print("no max rank")

def defPos(A):
    autoVal = np.linalg.eigvals(A)
    flag = autoVal > 0
    if np.all(flag):
        print("definita positiva")
    else:
        print("non definita positiva")

def simmetria(A):
    flag =np.array_equal(A.T , A)

    if flag:
        print("simmetrica")
        defPos(A)
    else:
        print("non simmetrica")

def diagExtreme(A):
    if np.all(np.diag(A) > np.abs(A-np.diag(A))):
        print("diag strettamente dominante")
    else:
        print("diagonale non strettamente dominante")

def checkSparsaOrDensa(A):
    non_zeros = np.count_nonzero(A)
    perc = non_zeros/(A.shape[0]*A.shape[1])*100
    print("perc non zeros => ",perc)
    if perc <= 33:
        print("matrice sparsa")
    else:
        print("matrice densa")


def getInfo(A):
    print(A.shape[0], A.shape[1])
    if A.shape[0] == A.shape[1]:
        print("quadrata")
    if min(A.shape) >= 200:
        print("matrice di grande dimensioni")
    else:
        print("matrice di piccole dimensioni")
    
    cond(A)
    checkSparsaOrDensa(A)
    checkMaxRank(A)
    simmetria(A)
    diagExtreme(A)
    print("---------------")

In [383]:
from scipy.io import loadmat
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt
import scipy as sp
import scipy.linalg as spl
import RisolviSis

dati = loadmat('Test_II.mat')
A1=dati["A1"]
A1=A1.astype(float)
b1=dati["b1"]
b1=b1.astype(float)
A2=dati["A2"]
A2=A2.astype(float)
b2=dati["b2"]
b2=b2.astype(float)
A3=dati["A3"]
A3=A3.astype(float)
b3=dati["b3"]
b3=b3.astype(float)

getInfo(A1)
getInfo(A2)
getInfo(A3)
getInfo(b3)

15 15
quadrata
matrice di piccole dimensioni
matrice mal condizionata
perc non zeros =>  100.0
matrice densa
max rank
simmetrica
definita positiva
diag strettamente dominante
---------------
15 6
matrice di piccole dimensioni
matrice ben condizionata
perc non zeros =>  100.0
matrice densa
max rank
non simmetrica
diagonale non strettamente dominante
---------------
225 225
quadrata
matrice di grande dimensioni
matrice ben condizionata
perc non zeros =>  2.528395061728395
matrice sparsa
max rank
non simmetrica
diagonale non strettamente dominante
---------------
225 1
matrice di piccole dimensioni
matrice ben condizionata
perc non zeros =>  100.0
matrice densa
max rank
non simmetrica
diag strettamente dominante
---------------


1. per il primo sistema utilizzo la fattorizzazione QR
2. per il secondo sistema utilizzo il metodo delle equazioni normali
3. per il terzo sistema utilizzo il metodo di Gauss_Seidel

In [384]:
import RisolviSis as solver
import scipy.linalg as spli
import numpy.linalg as npl
def eqnorm(A,b):
#Risolve un sistema sovradeterminato con il metodo delle equazioni normali
    G= A.T@A
     
    f= np.dot(A.T,b)
    
    L= spli.cholesky(G,lower=True)
    U=L.T
        
   
    z,flag=solver.Lsolve(L,f)
    if flag == 0:
        x,flag=solver.Usolve(U,z)
        return x
    

def QR(A,b):

    Q,R = np.linalg.qr(A)

    y = np.dot(Q.T,b)

    x = np.linalg.solve(R,y)

    return x

def gauss_seidel(A,b,x0,toll,it_max):
    errore=1000
    d=np.diag(A)
    D=np.diag(d)
    E=np.tril(A,-1)
    F=np.triu(A,1)
    M=(D+E)
    N=-F
    T=np.dot(np.linalg.inv(M),N)
    autovalori=np.linalg.eigvals(T)
    raggiospettrale=np.max(np.abs(autovalori))
    print("raggio spettrale Gauss-Seidel ",raggiospettrale)
    it=0
    er_vet=[]
    while it<it_max and errore>toll:
        temp=b-F@x0
        x,flag= RisolviSis.Lsolve(M,temp)
        errore= npl.norm(x-x0)/npl.norm(x)
        er_vet.append(errore)
        x0=x.copy()
        it=it+1
    return x,it,er_vet


In [385]:
x1 = QR(A1,b1)
x2 = eqnorm(A2,b2)
x0 = np.zeros((A3.shape[0],1))
x3,it,error_arr = gauss_seidel(A3,b3,x0,10e-8,1000)
print("qr => ",x1)
print("eqnorm =>",x2)
print("gauss-seidel => ",x3)

raggio spettrale Gauss-Seidel  0.024750078276735976
qr =>  [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
eqnorm => [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
gauss-seidel =>  [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]


In [386]:
print(" la matrice A1 è mal condizionata, questo vuol dire che anche un piccola perturbazione \nnei dati può portare ad un risultato molto diverso da quello che ci si aspetta")
A1_pert[0,0] = A1[0,0] + A1[0,0]*0.001
x1_pert = QR(A1_pert,b1)  
print("qr perturbato => ",x1_pert)
print("mentre le altre due matrici sono l'opposto di A1 ")
A2_pert = A2.copy()
A2_pert[0,0] = A2[0,0] + A2[0,0]*0.001
x2_pert = eqnorm(A2_pert,b2)
print("eqnorm perturbato =>",x2_pert)   
A3_pert = A3.copy()
A3_pert[0,0] = A3[0,0] + A3[0,0]*0.001
x0 = np.zeros((A3.shape[0],1))
x3_pert,it,error_arr = gauss_seidel(A3_pert,b3,x0,10e-8,1000)
print("gauss-seidel perturbato => ",x3_pert)
print("come possiamo vedere queste due matrici perturbate non cambiano di molto il risultato che ci aspettavamo")

 la matrice A1 è mal condizionata, questo vuol dire che anche un piccola perturbazione 
nei dati può portare ad un risultato molto diverso da quello che ci si aspetta
qr perturbato =>  [[ 0.00912177]
 [ 1.69660199]
 [ 1.46140203]
 [ 1.2206698 ]
 [ 0.81564148]
 [ 2.06601753]
 [ 3.05486309]
 [-1.23865134]
 [-0.45933569]
 [ 0.57283859]
 [ 0.95772835]
 [ 2.15884554]
 [ 1.47758937]
 [ 0.97612019]
 [ 0.58807521]]
mentre le altre due matrici sono l'opposto di A1 
eqnorm perturbato => [[0.99968906]
 [1.00020445]
 [1.00013808]
 [1.00002837]
 [1.00008876]
 [1.00007642]]
raggio spettrale Gauss-Seidel  0.024750078081630808
gauss-seidel perturbato =>  [[0.99900031]
 [0.99998147]
 [0.99999965]
 [0.99999999]
 [1.00000001]
 [1.00000034]
 [1.00000002]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [0.99998147]
 [0.99999931]
 [0.99999998]
 [1.        ]
 [1.        ]
 [1.00000001]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1

# Esercizio 2

Data l'equazione di 2° grado $$\frac{1}{2} x^2+2bx-c=0$$
che diventa $$x^2+4bx-2c=0$$

con $b=10^7, c=10^{-i},i=-5,\cdots,12 $

- a) Scrivere le due formule algebriche per ricavare i valori delle 2 soluzioni

                                                                                    1 punto
- b) Studiare l'indice di condizionamento delle 2 formule algebriche, facendo uso del risultato teorico visto a lezione riguardo l'indice di condizionamento della valutazione di una funzione. (Spiegare il significato di problema mal condizionato) e dire quale delle due formule è malcondizionata. Per quali valori di $c$ il problema risulta ben condizionato?

                                                                                    5 punti

- c) Nel caso in cui una delle due formule risulti mal condizionata, proporre una soluzione algebricamente equivalente che non sia malcondizionata.

                                                                                    3 punti

                                                                        Totale 9 punti

In [388]:
b = 10**7
i = np.arange(-5,13)
c = 10.0**(-i)

delta = lambda : 4*b**2+2*c

f = lambda x:x**2+4*b*x-2*c
fp = lambda x: 2*x+4*b

x1 = -2*b+np.sqrt(delta())
x2 = -2*b-np.sqrt(delta())
print(x1)
print(x2)

# ora calcolo gli indici di condizionamento della funzione dati dalla formula:
# K = f'(x)*x / f(x) 
# sostituendo ottengo che k1 è uguale a :
k1 = lambda: (-4*b+2*np.sqrt(4*b**2+2*c))/(4*b)

print(k1())
# come possiamo vedere con b grandi tende a 0
k2 = lambda:(2*b+np.sqrt(4*b**2+2*c))/(-2*b)
print(k2())


[4.99999896e-03 5.00001013e-04 5.00008464e-05 4.99933958e-06
 4.99188900e-07 4.84287739e-08 3.72529030e-09 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
[-40000000.005      -40000000.0005     -40000000.00005
 -40000000.000005   -40000000.0000005  -40000000.00000004
 -40000000.         -40000000.         -40000000.
 -40000000.         -40000000.         -40000000.
 -40000000.         -40000000.         -40000000.
 -40000000.         -40000000.         -40000000.        ]
[2.49999948e-10 2.50000507e-11 2.50004232e-12 2.49966979e-13
 2.49594450e-14 2.42143869e-15 1.86264515e-16 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
[-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]


## Domanda intelligenza artificiale

 Limiti delle reti MLP (multilayer Perceptron) ed introduzione delle Reti neurali Convoluzionali. 
- Architettura di una rete neurale convoluzionale: strati convoluzionali, attivazione non lineare e pooling + parte fully connected.
- Loss function per il task della regressione.  Training di una rete. 
- Cenni sull'algoritmo di backpropagation per il calcolo delle derivate parziale della funzione costo rispetto ai pesi ditutti i layer .
- Tecniche di Ottimizzazione: metodo di discesa del gradient batch, metodo del gradiente stocastico (SGD) ,metodo del gradiente stocastico minibatch.
                                                                                [7  punti]
